In [1]:
import yaml
import boto3
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [2]:
import os
os.getcwd()

'/home/ec2-user/SageMaker/sisyphus/AthenaInsights/notebooks/data_cleaning'

In [3]:
client = boto3.client('s3')

In [4]:
s3_prefix = 's3://'
bucket = 'sisyphus-general-bucket'
primary_folder = 'AthenaInsights'

In [5]:
paginator = client.get_paginator('list_objects_v2')

# List objects under the specified prefix
page_iterator = paginator.paginate(Bucket=bucket, Prefix='AthenaInsights/latest_data/', Delimiter='/')

for page in page_iterator:
    if 'CommonPrefixes' in page:
        for folder in page['CommonPrefixes']:
            print(folder['Prefix'])

AthenaInsights/latest_data/csv/
AthenaInsights/latest_data/data_prep/
AthenaInsights/latest_data/dependent_variable/
AthenaInsights/latest_data/feature_prep/
AthenaInsights/latest_data/parquet/
AthenaInsights/latest_data/reduced_autocorelation/


### dependent variable

In [6]:
client = boto3.client('s3')

In [7]:
s3_prefix = 's3://'
bucket = 'sisyphus-general-bucket'
primary_folder = 'AthenaInsights'

In [8]:
# dependent_variable_loc = 's3://sisyphus-general-bucket/AthenaInsights/latest_data/dependent_variable/stock_bars_1min.parquet'
# dependent_variable = pd.read_parquet(dependent_variable_loc, columns=['category', 'close_sma_5m' ])

### all paths

In [9]:
paginator = client.get_paginator('list_objects_v2')

# List objects under the specified prefix
page_iterator = paginator.paginate(Bucket=bucket, Prefix='AthenaInsights/latest_data/', Delimiter='/')

for page in page_iterator:
    if 'CommonPrefixes' in page:
        for folder in page['CommonPrefixes']:
            print(folder['Prefix'])

AthenaInsights/latest_data/csv/
AthenaInsights/latest_data/data_prep/
AthenaInsights/latest_data/dependent_variable/
AthenaInsights/latest_data/feature_prep/
AthenaInsights/latest_data/parquet/
AthenaInsights/latest_data/reduced_autocorelation/


In [10]:
s3_prefix = 's3://'
bucket = 'sisyphus-general-bucket'
primary_folder = 'AthenaInsights'

In [11]:
response = client.list_objects_v2(
    Bucket=bucket,
    Prefix=f'{primary_folder}/latest_data/feature_prep/')

In [12]:
paths = []
for content in response.get('Contents', []):
    # print(f"{s3_prefix}{bucket}/{content['Key']}")
    k = content['Key']
    if 'base' in k and '1min' in k: #('base' in k or 'diff' in k) and ('rsi' not in k and 'macd' not in k):
        paths.append(f"{s3_prefix}{bucket}/{content['Key']}")
paths

['s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base_macd.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base_other_features.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base_rsi.parquet']

In [13]:
paths = []
for content in response.get('Contents', []):
    # print(f"{s3_prefix}{bucket}/{content['Key']}")
    k = content['Key']
    if 'base' in k and '2min' in k: #('base' in k or 'diff' in k) and ('rsi' not in k and 'macd' not in k):
        paths.append(f"{s3_prefix}{bucket}/{content['Key']}")
paths

['s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_2min_base.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_2min_base_macd.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_2min_base_rsi.parquet']

### independent variables

In [14]:
paths = []
for content in response.get('Contents', []):
    # print(f"{s3_prefix}{bucket}/{content['Key']}")
    k = content['Key']
    paths.append(f"{s3_prefix}{bucket}/{content['Key']}")

In [15]:
len(paths)

151

In [16]:
import pyarrow.dataset as ds
from tqdm import tqdm

In [17]:
# names = []
# time_periods = []
# from pyarrow.parquet import read_metadata
# x = read_metadata(path)
# dir(x)
# x.schema[0].path

In [18]:
all_info = pd.DataFrame(columns=['name', 'time_period', 'path', 'cols'])

for path in tqdm(paths):
    name = path.split('/')[-1].replace('.parquet', '')
    time_period = name.replace('stock_bars_', '').split('_')[0]
    df = ds.dataset(path).scanner().head(1).to_pandas()
    columns = list(df.columns)
    all_info.loc[all_info.shape[0]] = [name, time_period, path, columns]

100%|██████████| 151/151 [01:29<00:00,  1.68it/s]


In [19]:
all_info.head()

,name,time_period,path,cols
0,stock_bars_100D_base,100D,s3://sisyphus-general-bucket/AthenaInsights/la...,"[open, high, low, close, volume, symbol, time_..."
1,stock_bars_100D_base_macd,100D,s3://sisyphus-general-bucket/AthenaInsights/la...,"[symbol, close_ema_5m, close_ema_8m, close_ema..."
2,stock_bars_100D_base_rsi,100D,s3://sisyphus-general-bucket/AthenaInsights/la...,"[close, close_ema_5m, close_sma_5m, close_ema_..."
3,stock_bars_100D_diff,100D,s3://sisyphus-general-bucket/AthenaInsights/la...,"[open_diff, high_diff, low_diff, close_diff, v..."
4,stock_bars_100D_diff_macd,100D,s3://sisyphus-general-bucket/AthenaInsights/la...,"[symbol, close_diff_ema_5m, close_diff_ema_8m,..."


In [20]:
paths_config = dict().fromkeys(list(all_info.time_period.unique()), dict())
for row in all_info.iterrows():
    paths_config[time_period][row[1]['name']] = {}
    paths_config[time_period][row[1]['name']]['path'] = row[1]['path']
    paths_config[time_period][row[1]['name']]['cols'] = row[1]['cols']

In [21]:
paths_config.keys()
paths_config['100D'].keys()
paths_config['100D']['stock_bars_100D_base'].keys()

dict_keys(['path', 'cols'])

In [23]:
with open('../../config/spy_30min_v1/features.yaml', 'w+') as ff:
    yaml.dump(paths_config, ff)